# Main code for online regression on Jupyter notebooks

About: predicting the beer consumation of the Brazilian population

Related files:

Instructions:

In [4]:
# Preparar o Dataset de treinamento do modelo inicial OK
# Preparar o Dataset para treinamentos extrasOK
# Salvar cada um em Arquivo do Excel OK
# Preparar arquivo excel para suporte de variáveis
data_in = 0
# preparar arquivo do excel para Dataset outlyers

In [12]:
# Downloads
# import ...
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import pandas as pd
import numpy as np
import seaborn as sns
import pickle
from ipywidgets import widgets, HBox, VBox
from IPython.display import display
from IPython.display import clear_output

# Read Dataset renda
beer_data_to_train = pd.read_csv("beer_data_train", sep = ',')
# Read Dataset variáveis
beer_data_update = pd.read_csv("beer_data_update", sep = ',')

accumulated_cases = 0
temp_data = pd.DataFrame(data=None, columns=beer_data_to_train.columns, index=None) 


In [6]:
# Criação do modelo
# Cria modelo
model_main = LinearRegression()
# Prepara Dataset de Treino
y = beer_data_to_train['consumo']
X = beer_data_to_train[['temp_max', 'chuva', 'fds']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2811)
# Treina o modelo
model_main.fit(X_train, y_train)
# Testa o modelo
y_predicted = model_main.predict(X_test)
print('R² for training data = {}'.format(model_main.score(X_train, y_train).round(3)))
R_model = metrics.r2_score(y_test, y_predicted)
print('R² for tested data = %s' %R_model.round(3))
# Salva o modelo
output = open('beer_consumption_model', 'wb') #cria o arquivo
pickle.dump(model_main, output) #salva o modelo no arquivo
output.close()

R² for training data = 0.711
R² for tested data = 0.772


In [7]:
# Button 1
# Opção de digitar 1 caso para o modelo prever
temp_max1 = widgets.Text(description="Max Temp")
rain1 = widgets.Text(description="Rain")
weekend1 = widgets.Text(description="Weekend")
button1 = widgets.Button(description="Predict")

def simulator1(sender):
    entry=[[
                float(temp_max1.value if temp_max1.value else 0), 
                float(rain1.value if rain1.value else 0), 
                float(weekend1.value if weekend1.value else 0)                
             ]]
    print('{0:.2f} liters'.format(model_main.predict(entry)[0]))

top = VBox([temp_max1, rain1, weekend1])
button1.on_click(simulator1) 
top1 = HBox([top, button1])

In [18]:
# Button 2
temp_max2 = widgets.Text(description="Max Temp")
rain2 = widgets.Text(description="Rain")
weekend2 = widgets.Text(description="Weekend")
beer = widgets.Text(description="Sold beer")
button2 = widgets.Button(description="Add new data")

def simulator2(sender):
    global accumulated_cases, model_main, R_model, beer_data_to_train, temp_data
    entry=[[
                float(temp_max2.value if temp_max2.value else 0), 
                float(rain2.value if rain2.value else 0), 
                float(weekend2.value if weekend2.value else 0),
                float(beer.value if beer.value else 0)
             ]]
    data_entry = pd.DataFrame(data=entry, columns=temp_data.columns, index=None) 
    temp_data = temp_data.append(data_entry)
    accumulated_cases += 1 
    print("Data to update on the model:")
    print(temp_data)
    #print(temp_data)
    #print(data_entry)
    if accumulated_cases >= 3:
        [model_main, R_model] = retrain(beer_data_to_train,temp_data)
        temp_data = pd.DataFrame(data=None, columns=beer_data_to_train.columns, index=None) 
        accumulated_cases = 0
     
middle = VBox([temp_max2, rain2, weekend2, beer])
button2.on_click(simulator2) 
middle1 = HBox([middle,button2])


SyntaxError: invalid syntax (Temp/ipykernel_17944/2749676426.py, line 2)

In [9]:
# Button3 add3 (adiciona 3 casos do outro dataset, e treina novamente)
button3 = widgets.Button(description="3 new cases")

#def simulator3(sender):
    #Retreina
    
#button3.on_click(simulator3) 


In [17]:
# Display
inputs1 = VBox([top1])
inputs2 = VBox([middle1])

display(inputs1, inputs2)#, button3)

Data to update on the model:
   temp_max  chuva  fds  consumo
0       0.0    0.0  0.0      0.0
0       0.0    0.0  0.0      0.0
Data to update on the model:
   temp_max  chuva  fds  consumo
0       0.0    0.0  0.0      0.0
0       0.0    0.0  0.0      0.0
0       0.0    0.0  0.0      0.0
Training new model...
testing new model...
The New model isnt a better model, discarting changings


In [19]:
# Retreinar modelo
def retrain(df1,df2):
    global model_main, R_model
    print('Training new model...')
    beer_data_temp = df1.append(df2)
    # Cria modelo2
    model_temp = LinearRegression()
    # treina modelo2
    y2 = beer_data_temp['consumo']
    X2 = beer_data_temp[['temp_max', 'chuva', 'fds']]
    X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.3, random_state=2811)
    model_temp.fit(X2_train, y2_train)
    # compara resultado modelo2 com modelo1, e se melhor
    # substitui modelo1, se pior, coloca os dados na planilha 3
    print('testing new model...')
    y2_predicted = model_temp.predict(X2_test)
    
    R_temp = metrics.r2_score(y2_test, y2_predicted)
    
    if R_temp > R_model:
        print('We trained a new model with the data, the new R2 is {0:.2f}'.format(R_model.round(4)))
        R_model, model_main  = R_temp, model_temp
    else:
        print('The New model isnt a better model, discarting changings')
        #Mantem o mesmo modelo, manda os dados para uma coluna de outlyers
    return [model_main, R_model]